In [1]:
import numpy as np, torch, os, multiprocessing, pandas as pd, sys
import torch.backends.cudnn as cudnn, torchio as tio, random
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
from sklearn.metrics import *
sys.path.append("..")
from utils.model_res import generate_model

from tqdm import tqdm   

num_workers = multiprocessing.cpu_count()
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True
random.seed(1234)
torch.manual_seed(1234)

In [2]:
# load csv label
if True:
    csv_path = './20211104_label_1-350.csv'
    table =  pd.read_csv(csv_path)
    table_3t = table[table['1/0: 3T/1.5T MRI']==1.0]
    table_3t_test = table_3t[table_3t['Valid data']=='V']
    table_3t_test = np.array(table_3t_test[table_3t_test['排除']=='Test data'])
    nii_3t_test = sorted([i for i in os.listdir(os.path.join('./dataset/S2_data/','test'))])

In [3]:
# Subject Function Building
def tio_process(nii_3t_, table_3t_, basepath_='./dataset/S2_data/train/'):
    subjects_ = []
    for  (nii_path, nii_table) in zip(nii_3t_ , table_3t_):
        if (params['S2_type']=='ap') and (nii_table[3]=='A' or nii_table[3]=='P'):
            subject = tio.Subject(
                dwi = tio.ScalarImage(os.path.join(basepath_, nii_path)), 
                ap = nii_table[3], 
                score=[])
            subjects_.append(subject)
        elif (params['S2_type']=='nl'):
            subject = tio.Subject(
                dwi = tio.ScalarImage(os.path.join(basepath_, nii_path)), 
                nl  = nii_table[4], 
                score=[])
            subjects_.append(subject)
    return subjects_


In [4]:
def change_subject_img(subject_, tesnor_ary):
    for idx, i in enumerate(subject_):
        if False:
            print( subject_[idx]['dwi'].shape)
            for idx2 in range(26):
                fig = plt.figure(figsize=(12,12))
                ax1 = fig.add_subplot(1,3,1)
                ax1.imshow(np.squeeze((subject_[idx]['dwi'][tio.DATA])[...,idx2], axis=0), cmap='bone')
                ax1.get_xaxis().set_visible(False)
                ax1.get_yaxis().set_visible(False)
                ax2 = fig.add_subplot(1,3,2)
                ax2.imshow(np.squeeze((tesnor_ary[idx])[...,idx2], axis=0), cmap='bone')
                ax2.get_xaxis().set_visible(False)
                ax2.get_yaxis().set_visible(False)
                plt.show()
                plt.close(fig)
        subject_[idx]['dwi'].set_data(tesnor_ary[idx])
        # image =  subject_[idx]['dwi'][tio.DATA]
    return subject_

def label2value(label):
    if params["S2_type"]=='nl':
        target = [0 if i=='N' else 1 for i in label]
    else:
        target = [0 if i=='A' else 1 for i in label]
    return torch.LongTensor(target).to(device)

def S2_evaluate(model,valid_loader):
    predict_array = {'target':[], 'predict':[]}
    S2_model.eval()
    stream_v = tqdm(valid_loader)
    with torch.no_grad():
        for i, data in enumerate(stream_v, start=1):
            # print(data['dwi']['path'])
            if False:
                for idx2 in range(26):
                    fig = plt.figure(figsize=(12,12))
                    ax1 = fig.add_subplot(1,1,1)
                    ax1.imshow(np.squeeze((data['dwi'][tio.DATA])[...,idx2], axis=0), cmap='bone')
                    ax1.get_xaxis().set_visible(False)
                    ax1.get_yaxis().set_visible(False)
                    plt.show()
                    plt.close(fig)
            images = data['dwi'][tio.DATA].to(device)
            target = label2value(data[params["S2_type"]])
            images = images.to(device)
            target = target.to(device)
            output = model(images).squeeze(1)
            output = torch.sigmoid(output)
            _, outputs = torch.max(output, 1)
            predict_array['predict'].append(outputs.item())
            predict_array['target'].append(target.item())
    return predict_array
if True:
    test_transform = tio.Compose([])
    S2_weight_ap = './checkpoint/2021.11.16.t1 - 3DResNet18 - ap/best - ap - 3dresnet18 - lr_0.001 - CE.pt'
    S2_weight_nl = './checkpoint/2021.11.16.t2 - 3DResNet18 - nl/best - vloss - nl - 3dresnet18 - lr_0.001 - FL.pt'
    S2_weight_stack = {'ap': S2_weight_ap, 'nl': S2_weight_nl}
    S2_reply = {'ap': [], 'nl':[]}
for idx, i in enumerate(['ap', 'nl']):
# for idx, i in enumerate(['nl']):
    # ///////////////////////////////////////////////////
    S2_model = generate_model(model_depth=18, n_input_channels=1, n_classes=2)
    S2_checkpoint= torch.load(S2_weight_stack[i], map_location=torch.device(device))
    S2_model.load_state_dict(S2_checkpoint['model_state_dict'])
    S2_model.to(device)
    # //////////////////////////////////////////////////
    params = {"S1_type": None, "S2_type": i}
    S2_subjects = tio_process(nii_3t_test, table_3t_test, basepath_ = './dataset/S2_data/test/')
    S2_set = tio.SubjectsDataset(S2_subjects, transform=test_transform)
    test_loader2 = torch.utils.data.DataLoader(S2_set, batch_size=1, shuffle=False, num_workers=4)
    S2_reply[i].append(S2_evaluate(S2_model, test_loader2))



100%|██████████| 45/45 [01:52<00:00,  2.49s/it]


In [54]:
for i in ['ap', 'nl']:
    GT =np.array(S2_reply[i][0]['target'])
    SR = np.array(S2_reply[i][0]['predict'])
    TP = int((SR * GT).sum()) #TP
    FN = int((GT * (1-SR)).sum()) #FN
    TN = int(((1-GT) * (1-SR)).sum()) #TN
    FP = int(((1-GT) * SR).sum()) #FP
    print(f'{i}  -   Accuracy  :', round((TP + TN)/(TP + TN + FP + FN), 2)*100, '%')
    print(f'{i}  -   Sensitivity  :', round(float(TP)/(float(TP+FN) + 1e-6), 5))
    print(f'{i}  -   Specificity  :', round(float(TN)/(float(TN+FP) + 1e-6), 5))

ap  -   Accuracy  : 93.0 %
ap  -   Sensitivity  : 0.85714
ap  -   Specificity  : 0.96552
nl  -   Accuracy  : 91.0 %
nl  -   Sensitivity  : 1.0
nl  -   Specificity  : 0.80952
